# OMOP Data Exploration and Analysis with PostgreSQL

This jupyter notebook includes a example for querying and analyzing OMOP(Observational Medical Outcomes Partnership) data stored in a PostgreSQL database. The notebook covers essential steps, from establishing a connection to the database to executing SQL queries, and visualizing key insights.

https://www.ohdsi.org/data-standardization/

### Importing required Libraries

- **psycopg2** For establishing a connection between python and a PostgreSQL database.
- **pandas** For efficient data handling, manipulation nad analysis of data.
- **sqlio** For executing SQL queries and reading the results directly into pandas dataframe.

In [1]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2 as ps
import warnings

warnings.filterwarnings('ignore') #ignoring warning related to sqlio and sql alchemy. Will improve this.

### Database connection and setup

Please change the username password to your OMOP DB.

In [2]:
conn = ps.connect(dbname="ohdsi",
                   user = "ohdsi_admin_user",
                   password = "admin1",
                   host = "omop-db-postgress",
                   port = "5432")

## Initial Analysis - Query Counts from OMOP DB
### Number of Person

In [5]:
df = sqlio.read_sql_query("SELECT COUNT(*) FROM cds_cdm.condition_occurrence", conn)
df.head(60)

,count
0,0


### Number of Observation

In [6]:
df = sqlio.read_sql_query("SELECT * FROM cds_cdm.observation", conn)
df.head(100)

,observation_id,person_id,observation_concept_id,observation_date,observation_datetime,observation_type_concept_id,value_as_number,value_as_string,value_as_concept_id,qualifier_concept_id,unit_concept_id,provider_id,visit_occurrence_id,visit_detail_id,observation_source_value,observation_source_concept_id,unit_source_value,qualifier_source_value,fhir_identifier,fhir_logical_id
0,2,3,0,2023-11-27,2023-11-27 18:40:02,32883,NaN,IPRD.DE40,80000001.0,NaN,NaN,None,None,None,IPRD.DE39,70000077,None,None,None,obs-754207d5-f3e7-406d-b750-9acc8f9b71f4
1,11,5,0,2023-12-04,2023-12-04 12:46:29,32817,300.0,None,NaN,NaN,NaN,None,None,None,ANC.B8.DE19,60000003,None,None,None,obs-77df47d1-e636-436d-8566-dd250f3dc67c
2,15,7,0,2023-12-06,2023-12-06 13:54:09,32817,NaN,ANC.B9.DE27,0.0,NaN,NaN,None,None,None,ANC.B9.DE24,60000007,None,None,None,obs-ea525104-beb8-48e6-af60-a113a2e7a42f
3,19,9,0,2024-01-08,2024-01-08 19:26:49,32817,NaN,IPRD.DE192,80000018.0,NaN,NaN,None,None,None,IPRD.DE188,70000082,None,None,None,obs-a48365ad-aba6-4db3-8469-10907905ff20
4,20,13,0,2024-01-10,2024-01-10 16:05:15,32883,NaN,IPRD.DE117,80000019.0,NaN,NaN,None,None,None,IPRD.DE116,70000054,None,None,None,obs-7dca0c24-7864-46d5-affb-694508cf0564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,180,30,0,2024-01-31,2024-01-31 18:21:09,32817,NaN,ANC.B7.DE.99,80000056.0,NaN,NaN,None,None,None,ANC-B7-DE98,70000034,None,None,None,obs-4c737c63-8fd3-45b1-a412-7ffd784033b0
96,181,30,0,2024-01-31,2024-01-31 18:30:29,32817,NaN,ANC.B7.DE.99,80000056.0,NaN,NaN,None,None,None,ANC-B7-DE98,70000034,None,None,None,obs-9bdc5302-e54f-4e5c-8768-609e9110f67f
97,184,18,0,2024-02-06,2024-02-06 12:05:31,32817,NaN,IPRD.DE91,80000031.0,NaN,NaN,None,None,None,IPRD.DE155,70000051,None,None,None,obs-877fd669-a082-4606-a94d-5c1aec7ffdf4
98,186,1,0,2018-02-14,2018-02-14 12:30:00,32856,148.0,None,NaN,4267416.0,8848.0,None,None,None,777-3,0,10*3/uL,L,obs-id-017d8dc1aaad9b53,obs-112a6e5d-12d6-402d-8956-8ae082964c4f


### Number of Death

In [9]:
df = sqlio.read_sql_query("SELECT COUNT(*) FROM cds_cdm.death", conn)
df.head()

,count
0,2


### Number of Procedure

### Number of measurements eg: Height, Weight etc.

In [10]:
df = sqlio.read_sql_query("SELECT COUNT(*) FROM cds_cdm.procedure_occurrence", conn)
df.head()

,count
0,13


In [1]:
#Number of Concepts
df = sqlio.read_sql_query("SELECT * FROM cds_cdm.concept", conn)
df.head(10)

NameError: name 'sqlio' is not defined

### Describe Height meaurements

In [9]:
df = sqlio.read_sql_query("SELECT * FROM cds_cdm.measurement where measurement_source_concept_id=3036277", conn)
df["value_as_number"].describe()

count       0
unique      0
top       NaN
freq      NaN
Name: value_as_number, dtype: object

## Encounter visit longitudinal Analysis

In [10]:
df = sqlio.read_sql_query(f"SELECT * FROM cds_cdm.visit_occurrence", conn)
df.head()

,visit_occurrence_id,person_id,visit_concept_id,visit_start_date,visit_start_datetime,visit_end_date,visit_end_datetime,visit_type_concept_id,provider_id,care_site_id,visit_source_value,visit_source_concept_id,admitting_source_concept_id,admitting_source_value,discharge_to_concept_id,discharge_to_source_value,preceding_visit_occurrence_id,fhir_identifier,fhir_logical_id


In [11]:
df = sqlio.read_sql_query(f"SELECT person_id, COUNT(person_id) AS count FROM cds_cdm.visit_occurrence GROUP BY person_id", conn)
df["count"].describe()

count       0
unique      0
top       NaN
freq      NaN
Name: count, dtype: object

### What has the person with max number of visits come for?

In [12]:
max_count = df["count"].describe().max() 
person_id_max_visit = df.loc[df['count'] == max_count]["person_id"].item()
print(person_id_max_visit)
df_new = sqlio.read_sql_query(f"SELECT * FROM cds_cdm.procedure_occurrence where person_id={person_id_max_visit}", conn)
procedure_concept_id = df_new['procedure_concept_id'].iat[0]
procedure_reason = sqlio.read_sql_query(f"SELECT * FROM cds_cdm.concept where concept_id={procedure_concept_id}", conn)
print(procedure_reason)

ValueError: can only convert an array of size 1 to a Python scalar

## Procedure specific longitudinal Analysis

### Enter OMOP code for a procedure eg: Mammography for breast cancer
#### Search example
https://athena.ohdsi.org/search-terms/terms?conceptClass=Procedure&page=1&pageSize=15&query=Mammography&boosts 

In [12]:
omop_code = 4324693 #Code for Mammography. Please change this to the code that you are interested in.
df = sqlio.read_sql_query(f"SELECT * FROM cds_cdm.procedure_occurrence where procedure_concept_id={omop_code}", conn)
df.head()

,procedure_occurrence_id,person_id,procedure_concept_id,procedure_date,procedure_datetime,procedure_type_concept_id,modifier_concept_id,quantity,provider_id,visit_occurrence_id,visit_detail_id,procedure_source_value,procedure_source_concept_id,modifier_source_value,fhir_identifier,fhir_logical_id


In [13]:
df = sqlio.read_sql_query(f"SELECT person_id, COUNT(person_id) AS count FROM cds_cdm.procedure_occurrence where procedure_concept_id={omop_code} GROUP BY person_id", conn)
df

,person_id,count
